# Day18_0 별첨: GAN (Generative Adversarial Networks)

## 📚 학습 목표

**이 별첨 자료에서는:**
1. GAN (Generative Adversarial Network)의 핵심 개념 이해하기
2. Generator와 Discriminator의 역할과 경쟁 구조 파악하기
3. GAN 손실 함수와 훈련 과정 이해하기
4. DCGAN (Deep Convolutional GAN) 구조 학습하기
5. MNIST 손글씨 이미지 생성 실습하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| GAN | 데이터 생성/증강 | 학습 데이터 부족 시 합성 데이터 생성 |
| Generator | 새로운 콘텐츠 생성 | 이미지, 음악, 텍스트 창작 |
| Discriminator | 품질 판별/필터링 | 가짜 이미지 탐지, 품질 평가 |
| DCGAN | 고품질 이미지 생성 | 얼굴 생성, 스타일 변환 |

**분석가 관점**: GAN은 '없는 데이터를 만들어내는' 혁신적 기술입니다. 데이터 증강, 이상치 생성, 시뮬레이션 등 데이터 부족 문제 해결에 활용됩니다. Stable Diffusion, DALL-E 등 최신 이미지 생성 AI의 기초 개념이기도 합니다!

---

# 1. GAN 개념

---

## 1.1 GAN이란?

**GAN (Generative Adversarial Network)**: 두 신경망이 서로 경쟁하며 학습하는 생성 모델

2014년 Ian Goodfellow가 제안한 혁신적인 아이디어로, '적대적 학습(Adversarial Training)'을 통해 실제와 구분하기 어려운 가짜 데이터를 생성합니다.

### 핵심 비유: 위조지폐범 vs 경찰

| 역할 | GAN 구성요소 | 목표 |
|------|-------------|------|
| 위조지폐범 | Generator (생성자) | 경찰도 속일 수 있는 진짜 같은 가짜 만들기 |
| 경찰 | Discriminator (판별자) | 진짜와 가짜를 정확히 구분하기 |

두 역할이 서로 경쟁하며 발전:
- 위조지폐범이 더 정교해지면 → 경찰도 더 날카로워짐
- 경찰이 더 정확해지면 → 위조지폐범도 더 정교해짐
- 결국 진짜와 구분 불가능한 수준에 도달!

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 재현성을 위한 시드 설정
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

## 1.2 GAN 구조

```
                    ┌─────────────────┐
   랜덤 노이즈 z ───→│   Generator     │───→ 가짜 이미지
                    └─────────────────┘           │
                                                  ↓
                                          ┌─────────────────┐
   진짜 이미지 ─────────────────────────→│  Discriminator  │───→ 진짜/가짜 판별
                                          └─────────────────┘
```

### Generator (생성자)
- **입력**: 랜덤 노이즈 벡터 z (예: 100차원 가우시안)
- **출력**: 가짜 이미지
- **목표**: Discriminator를 속일 수 있는 진짜 같은 이미지 생성

### Discriminator (판별자)
- **입력**: 이미지 (진짜 또는 가짜)
- **출력**: 진짜일 확률 (0~1)
- **목표**: 진짜와 가짜를 정확히 구분

---

# 2. GAN 손실 함수

---

## 2.1 Minimax Game

GAN의 학습은 **Minimax Game** (영합 게임)으로 표현됩니다:

$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}}[\log D(x)] + \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))]$$

### 각 항의 의미

| 항 | 의미 | Discriminator 관점 | Generator 관점 |
|---|------|-------------------|----------------|
| $\log D(x)$ | 진짜를 진짜로 판별 | 최대화 (1에 가깝게) | 관계없음 |
| $\log(1 - D(G(z)))$ | 가짜를 가짜로 판별 | 최대화 (0에 가깝게) | 최소화 (1에 가깝게) |

### 직관적 이해

- **Discriminator**: 진짜는 1, 가짜는 0으로 판별하고 싶음 → 손실 함수 최대화
- **Generator**: 가짜도 1로 판별되게 하고 싶음 → 손실 함수 최소화

In [ ]:
# GAN 손실 함수 시각화
d_output = np.linspace(0.01, 0.99, 100)

# 진짜 데이터에 대한 D의 손실: -log(D(x))
d_loss_real = -np.log(d_output)

# 가짜 데이터에 대한 D의 손실: -log(1 - D(G(z)))
d_loss_fake = -np.log(1 - d_output)

# G의 손실: -log(D(G(z))) [대안 형태]
g_loss = -np.log(d_output)

fig = make_subplots(rows=1, cols=2, subplot_titles=['Discriminator 손실', 'Generator 손실'])

# Discriminator 손실
fig.add_trace(
    go.Scatter(x=d_output, y=d_loss_real, name='진짜: -log(D(x))', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=d_output, y=d_loss_fake, name='가짜: -log(1-D(G(z)))', line=dict(color='red')),
    row=1, col=1
)

# Generator 손실
fig.add_trace(
    go.Scatter(x=d_output, y=g_loss, name='G 손실: -log(D(G(z)))', line=dict(color='green')),
    row=1, col=2
)

fig.update_xaxes(title_text='D(x) 또는 D(G(z))', row=1, col=1)
fig.update_xaxes(title_text='D(G(z))', row=1, col=2)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=2)

fig.update_layout(height=400, title_text='GAN 손실 함수 시각화')
fig.show()

print("💡 핵심 인사이트:")
print("- D가 진짜를 1로 판별할수록 → D 손실 감소 (파란선 왼쪽)")
print("- D가 가짜를 0으로 판별할수록 → D 손실 감소 (빨간선 왼쪽)")
print("- G가 가짜를 1로 속일수록 → G 손실 감소 (녹색선 오른쪽)")

## 2.2 실제 학습에서의 손실 함수

실제 구현에서는 Binary Cross Entropy (BCE)를 사용합니다:

```python
# Discriminator 손실
d_loss_real = BCE(D(real_images), ones)   # 진짜는 1로
d_loss_fake = BCE(D(G(z)), zeros)         # 가짜는 0으로
d_loss = d_loss_real + d_loss_fake

# Generator 손실 (Non-saturating 버전)
g_loss = BCE(D(G(z)), ones)               # 가짜를 1로 속이고 싶음
```

### Non-saturating Loss

원래 공식대로라면 G 손실은 $\log(1 - D(G(z)))$를 최소화해야 하지만,
학습 초기에 기울기가 거의 0이 되는 문제가 있어서 $-\log(D(G(z)))$를 최소화하는 방식을 주로 사용합니다.

---

# 3. 기본 GAN 구현

---

MNIST 손글씨 생성을 위한 기본 GAN을 구현해봅시다.

## 3.1 데이터 준비

In [ ]:
# MNIST 데이터셋 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # [-1, 1] 범위로 정규화
])

train_dataset = datasets.MNIST(
    root='./datasets',
    train=True,
    transform=transform,
    download=True
)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print(f"훈련 샘플 수: {len(train_dataset):,}")
print(f"배치 수: {len(train_loader):,}")
print(f"이미지 크기: {train_dataset[0][0].shape}")

## 3.2 Generator 정의

In [ ]:
class Generator(nn.Module):
    """
    Generator: 랜덤 노이즈를 이미지로 변환
    
    - 입력: 잠재 벡터 z (latent_dim 차원)
    - 출력: 생성된 이미지 (1, 28, 28)
    """
    def __init__(self, latent_dim=100, img_shape=(1, 28, 28)):
        super().__init__()
        self.img_shape = img_shape
        img_size = int(np.prod(img_shape))  # 1 * 28 * 28 = 784
        
        self.model = nn.Sequential(
            # 입력: (batch, latent_dim)
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, img_size),
            nn.Tanh()  # 출력 범위 [-1, 1]
        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

# 테스트
latent_dim = 100
generator = Generator(latent_dim=latent_dim)

# 랜덤 노이즈 생성
z = torch.randn(4, latent_dim)
fake_images = generator(z)

print(f"입력 노이즈 shape: {z.shape}")
print(f"생성 이미지 shape: {fake_images.shape}")
print(f"출력 범위: [{fake_images.min():.2f}, {fake_images.max():.2f}]")

## 3.3 Discriminator 정의

In [ ]:
class Discriminator(nn.Module):
    """
    Discriminator: 이미지가 진짜인지 가짜인지 판별
    
    - 입력: 이미지 (1, 28, 28)
    - 출력: 진짜일 확률 (0~1)
    """
    def __init__(self, img_shape=(1, 28, 28)):
        super().__init__()
        img_size = int(np.prod(img_shape))  # 784
        
        self.model = nn.Sequential(
            # 입력: (batch, 784)
            nn.Linear(img_size, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid()  # 출력 범위 [0, 1]
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)  # Flatten
        validity = self.model(img_flat)
        return validity

# 테스트
discriminator = Discriminator()

# 진짜 이미지와 가짜 이미지 판별
real_img, _ = train_dataset[0]
real_img = real_img.unsqueeze(0)  # 배치 차원 추가

real_pred = discriminator(real_img)
fake_pred = discriminator(fake_images)

print(f"진짜 이미지 판별: {real_pred.item():.4f}")
print(f"가짜 이미지 판별: {fake_pred.mean().item():.4f}")
print("(학습 전이라 랜덤한 값)")

## 3.4 훈련 루프

In [ ]:
# 모델 초기화
latent_dim = 100
generator = Generator(latent_dim=latent_dim).to(device)
discriminator = Discriminator().to(device)

# 옵티마이저
lr = 0.0002
betas = (0.5, 0.999)  # Adam의 모멘텀 파라미터 (GAN에서 일반적으로 사용)

optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=betas)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=betas)

# 손실 함수
criterion = nn.BCELoss()

print(f"Generator 파라미터: {sum(p.numel() for p in generator.parameters()):,}")
print(f"Discriminator 파라미터: {sum(p.numel() for p in discriminator.parameters()):,}")

In [ ]:
def train_gan(generator, discriminator, train_loader, n_epochs=50, sample_interval=10):
    """
    GAN 훈련 함수
    """
    history = {'d_loss': [], 'g_loss': [], 'd_real': [], 'd_fake': []}
    fixed_z = torch.randn(64, latent_dim, device=device)  # 고정 노이즈 (시각화용)
    
    for epoch in range(1, n_epochs + 1):
        d_losses, g_losses = [], []
        d_reals, d_fakes = [], []
        
        for real_imgs, _ in train_loader:
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            # 라벨 생성
            real_labels = torch.ones(batch_size, 1, device=device)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            
            # ========================
            # 1. Discriminator 훈련
            # ========================
            optimizer_D.zero_grad()
            
            # 진짜 이미지 판별
            real_pred = discriminator(real_imgs)
            d_loss_real = criterion(real_pred, real_labels)
            
            # 가짜 이미지 생성 및 판별
            z = torch.randn(batch_size, latent_dim, device=device)
            fake_imgs = generator(z)
            fake_pred = discriminator(fake_imgs.detach())  # Generator 그래프 분리
            d_loss_fake = criterion(fake_pred, fake_labels)
            
            # 총 Discriminator 손실
            d_loss = (d_loss_real + d_loss_fake) / 2
            d_loss.backward()
            optimizer_D.step()
            
            # ========================
            # 2. Generator 훈련
            # ========================
            optimizer_G.zero_grad()
            
            # Generator는 Discriminator를 속이고 싶음
            fake_pred = discriminator(fake_imgs)
            g_loss = criterion(fake_pred, real_labels)  # 가짜를 진짜로 판별하게
            
            g_loss.backward()
            optimizer_G.step()
            
            # 기록
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            d_reals.append(real_pred.mean().item())
            d_fakes.append(fake_pred.mean().item())
        
        # 에폭 평균
        history['d_loss'].append(np.mean(d_losses))
        history['g_loss'].append(np.mean(g_losses))
        history['d_real'].append(np.mean(d_reals))
        history['d_fake'].append(np.mean(d_fakes))
        
        # 진행 상황 출력
        if epoch % sample_interval == 0 or epoch == 1:
            print(f"Epoch {epoch:3d}/{n_epochs} | "
                  f"D Loss: {history['d_loss'][-1]:.4f} | "
                  f"G Loss: {history['g_loss'][-1]:.4f} | "
                  f"D(real): {history['d_real'][-1]:.3f} | "
                  f"D(fake): {history['d_fake'][-1]:.3f}")
            
            # 생성 이미지 시각화
            generator.eval()
            with torch.no_grad():
                sample_imgs = generator(fixed_z).cpu()
            generator.train()
            
            # 8x8 그리드로 표시
            grid = make_grid(sample_imgs, nrow=8, normalize=True, value_range=(-1, 1))
            plt.figure(figsize=(8, 8))
            plt.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
            plt.title(f'Epoch {epoch}')
            plt.axis('off')
            plt.show()
    
    return history

In [ ]:
# 훈련 실행 (짧은 버전 - 전체 훈련은 시간이 오래 걸림)
print("GAN 훈련 시작!")
print("=" * 60)

# 실제 학습 시 n_epochs=100 이상 권장
history = train_gan(generator, discriminator, train_loader, n_epochs=30, sample_interval=10)

print("=" * 60)
print("훈련 완료!")

In [ ]:
# 학습 곡선 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['손실 함수', '판별 결과'])

# 손실
fig.add_trace(
    go.Scatter(y=history['d_loss'], name='Discriminator Loss', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=history['g_loss'], name='Generator Loss', line=dict(color='red')),
    row=1, col=1
)

# 판별 결과
fig.add_trace(
    go.Scatter(y=history['d_real'], name='D(real)', line=dict(color='green')),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=history['d_fake'], name='D(fake)', line=dict(color='orange')),
    row=1, col=2
)

fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_xaxes(title_text='Epoch', row=1, col=2)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Probability', row=1, col=2)

fig.update_layout(height=400, title_text='GAN 학습 곡선')
fig.show()

print("💡 해석:")
print("- D(real) ≈ 0.5, D(fake) ≈ 0.5: 이상적인 균형 상태 (구분 불가)")
print("- G Loss가 안정적이면서 D(fake)가 올라가면 좋은 학습")

---

# 4. DCGAN (Deep Convolutional GAN)

---

## 4.1 DCGAN이란?

**DCGAN (Deep Convolutional GAN)**: CNN을 활용한 GAN으로, 더 고품질의 이미지를 생성할 수 있습니다.

### 기본 GAN vs DCGAN

| 구분 | 기본 GAN | DCGAN |
|------|----------|-------|
| Generator | FC 레이어 | 전치 합성곱 (ConvTranspose2d) |
| Discriminator | FC 레이어 | 합성곱 (Conv2d) |
| 활성화 함수 | ReLU/LeakyReLU | Generator: ReLU, Discriminator: LeakyReLU |
| 정규화 | Dropout | BatchNorm |
| 이미지 품질 | 흐릿함 | 더 선명함|

### DCGAN의 핵심 가이드라인 (논문 제시)

1. Pooling 레이어 대신 Strided Convolution 사용
2. Generator와 Discriminator 모두 BatchNorm 사용
3. FC 레이어 최소화
4. Generator: ReLU + 마지막은 Tanh
5. Discriminator: LeakyReLU 사용

In [ ]:
class DCGenerator(nn.Module):
    """
    DCGAN Generator
    
    - ConvTranspose2d로 이미지 업샘플링
    - BatchNorm + ReLU
    - 마지막은 Tanh
    """
    def __init__(self, latent_dim=100, channels=1, feature_maps=64):
        super().__init__()
        self.latent_dim = latent_dim
        
        self.model = nn.Sequential(
            # 입력: (batch, latent_dim, 1, 1)
            # 출력: (batch, feature_maps*4, 4, 4)
            nn.ConvTranspose2d(latent_dim, feature_maps * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_maps * 4),
            nn.ReLU(True),
            
            # (batch, feature_maps*4, 4, 4) -> (batch, feature_maps*2, 7, 7)
            nn.ConvTranspose2d(feature_maps * 4, feature_maps * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 2),
            nn.ReLU(True),
            
            # (batch, feature_maps*2, 7, 7) -> (batch, feature_maps, 14, 14)
            nn.ConvTranspose2d(feature_maps * 2, feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps),
            nn.ReLU(True),
            
            # (batch, feature_maps, 14, 14) -> (batch, channels, 28, 28)
            nn.ConvTranspose2d(feature_maps, channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, z):
        # z: (batch, latent_dim) -> (batch, latent_dim, 1, 1)
        z = z.view(-1, self.latent_dim, 1, 1)
        return self.model(z)

# 테스트
dc_generator = DCGenerator(latent_dim=100)
z = torch.randn(4, 100)
fake_imgs = dc_generator(z)
print(f"DCGAN Generator 출력 shape: {fake_imgs.shape}")

In [ ]:
class DCDiscriminator(nn.Module):
    """
    DCGAN Discriminator
    
    - Conv2d로 이미지 다운샘플링
    - BatchNorm + LeakyReLU
    - 마지막은 Sigmoid
    """
    def __init__(self, channels=1, feature_maps=64):
        super().__init__()
        
        self.model = nn.Sequential(
            # 입력: (batch, channels, 28, 28)
            # 출력: (batch, feature_maps, 14, 14)
            nn.Conv2d(channels, feature_maps, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # (batch, feature_maps, 14, 14) -> (batch, feature_maps*2, 7, 7)
            nn.Conv2d(feature_maps, feature_maps * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # (batch, feature_maps*2, 7, 7) -> (batch, feature_maps*4, 4, 4)
            nn.Conv2d(feature_maps * 2, feature_maps * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # (batch, feature_maps*4, 4, 4) -> (batch, 1, 1, 1)
            nn.Conv2d(feature_maps * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        return self.model(img).view(-1, 1)

# 테스트
dc_discriminator = DCDiscriminator()
pred = dc_discriminator(fake_imgs)
print(f"DCGAN Discriminator 출력 shape: {pred.shape}")

## 4.2 가중치 초기화

DCGAN 논문에서 제안하는 가중치 초기화 방법입니다.

In [ ]:
def weights_init(m):
    """
    DCGAN 가중치 초기화
    - Conv/ConvTranspose: N(0, 0.02)
    - BatchNorm: N(1, 0.02), bias=0
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# DCGAN 모델 초기화
dc_generator = DCGenerator(latent_dim=100).to(device)
dc_discriminator = DCDiscriminator().to(device)

dc_generator.apply(weights_init)
dc_discriminator.apply(weights_init)

print(f"DCGAN Generator 파라미터: {sum(p.numel() for p in dc_generator.parameters()):,}")
print(f"DCGAN Discriminator 파라미터: {sum(p.numel() for p in dc_discriminator.parameters()):,}")

---

# 5. GAN의 문제점과 해결책

---

## 5.1 주요 문제점

| 문제 | 설명 | 증상 |
|------|------|------|
| **Mode Collapse** | Generator가 다양성 없이 몇 가지 샘플만 생성 | 모든 출력이 비슷함 |
| **훈련 불안정** | D와 G의 균형이 무너짐 | 손실이 진동하거나 발산 |
| **Vanishing Gradient** | D가 너무 잘 학습되면 G의 기울기가 0 | G가 학습되지 않음 |
| **평가의 어려움** | 생성 품질을 수치화하기 어려움 | 주관적 평가에 의존 |

## 5.2 해결책 및 개선 기법

| 문제 | 해결책 | 설명 |
|------|--------|------|
| Mode Collapse | Minibatch Discrimination | 배치 내 다양성 고려 |
| 훈련 불안정 | WGAN (Wasserstein GAN) | 더 안정적인 손실 함수 |
| Vanishing Gradient | Non-saturating Loss | $-\log(D(G(z)))$ 사용 |
| 평가 어려움 | FID (Fréchet Inception Distance) | 정량적 평가 지표 |

In [ ]:
# Mode Collapse 시각화 (시뮬레이션)
fig = make_subplots(rows=1, cols=2, subplot_titles=['정상 학습', 'Mode Collapse'])

# 정상 학습: 다양한 분포
normal_x = np.random.randn(500)
normal_y = np.random.randn(500)

# Mode Collapse: 몇 가지 모드에만 집중
collapse_centers = [(-2, -2), (2, 2), (0, 0)]
collapse_x, collapse_y = [], []
for cx, cy in collapse_centers:
    collapse_x.extend(np.random.randn(150) * 0.1 + cx)
    collapse_y.extend(np.random.randn(150) * 0.1 + cy)

fig.add_trace(
    go.Scatter(x=normal_x, y=normal_y, mode='markers', marker=dict(size=5, color='blue', opacity=0.5)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=collapse_x, y=collapse_y, mode='markers', marker=dict(size=5, color='red', opacity=0.5)),
    row=1, col=2
)

fig.update_layout(height=400, title_text='Mode Collapse 시각화', showlegend=False)
fig.show()

print("💡 Mode Collapse:")
print("- 왼쪽: 다양한 샘플 생성 (정상)")
print("- 오른쪽: 몇 가지 패턴만 반복 생성 (Mode Collapse)")

---

# 6. GAN의 응용 분야

---

## 6.1 다양한 GAN 변형

| GAN 변형 | 용도 | 특징 |
|----------|------|------|
| **CGAN** (Conditional) | 조건부 생성 | 라벨 조건으로 특정 클래스 생성 |
| **Pix2Pix** | 이미지-이미지 변환 | 스케치→사진, 흑백→컬러 |
| **CycleGAN** | 비쌍 이미지 변환 | 말→얼룩말, 여름→겨울 |
| **StyleGAN** | 고해상도 얼굴 생성 | This Person Does Not Exist |
| **SRGAN** | 초해상도 | 저해상도→고해상도 |

## 6.2 실무 활용 사례

| 분야 | 활용 | 예시 |
|------|------|------|
| 데이터 증강 | 학습 데이터 부족 해결 | 의료 영상, 희귀 케이스 |
| 이상 탐지 | 정상 데이터만으로 이상 감지 | 제조 불량 탐지 |
| 예술/창작 | AI 아트 생성 | NFT, 게임 에셋 |
| 개인정보 보호 | 합성 데이터 생성 | 개인정보 비식별화 |

---

## 📊 학습 정리

### 핵심 개념 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|------------|
| GAN | 생성자 vs 판별자 경쟁 | Minimax Game |
| Generator | 노이즈 → 이미지 | Discriminator 속이기 |
| Discriminator | 이미지 → 진위 판별 | 진짜/가짜 구분 |
| DCGAN | CNN 기반 GAN | 고품질 이미지 생성 |
| Mode Collapse | 다양성 손실 | 주요 문제점 |

### GAN vs 다른 생성 모델

| 특성 | GAN | VAE | Diffusion |
|------|-----|-----|-----------||
| 이미지 품질 | 높음 | 중간 | 매우 높음 |
| 훈련 안정성 | 낮음 | 높음 | 높음 |
| 다양성 | Mode Collapse 위험 | 좋음 | 매우 좋음 |
| 속도 | 빠름 | 빠름 | 느림 |

### 💡 실무 팁

1. **훈련 안정성**: 학습률을 낮게, BatchNorm 사용
2. **D/G 균형**: D가 너무 강하면 G 학습 불가 → D 업데이트 횟수 조절
3. **평가 지표**: FID, IS (Inception Score) 활용
4. **최신 트렌드**: Diffusion 모델이 GAN을 대체하는 추세
5. **실무 적용**: 데이터 증강, 합성 데이터 생성에 유용

---

## 추가 학습 자료

### 논문
- GAN 원본 논문: "Generative Adversarial Nets" (Goodfellow et al., 2014)
- DCGAN 논문: "Unsupervised Representation Learning with Deep Convolutional GANs" (Radford et al., 2015)

### 발전 방향
- **WGAN**: Wasserstein 거리를 사용한 안정적인 학습
- **Progressive GAN**: 점진적 해상도 증가
- **StyleGAN**: 스타일 기반 생성으로 고품질 얼굴 이미지
- **Diffusion Models**: GAN을 대체하는 새로운 패러다임 (Stable Diffusion, DALL-E)

**본 과정에서는 최신 트렌드인 LLM과 RAG에 집중합니다. GAN의 개념을 이해했다면, 필요 시 심화 학습을 권장합니다!**